In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm
from tqdm.auto import tqdm

import sys
sys.path.append('../utils')
from utils import *

In [2]:
combined_df = retrieve_data()
combined_df

,Date,ROG_Last_Price,ROG_IVOL,CFR_Last_Price,CFR_IVOL,ZURN_Last_Price,ZURN_IVOL,Risk_Free_Rate
0,2023-10-25,237.55,17.6858,107.00,33.0793,426.4,15.7900,1.4594
1,2023-10-26,237.25,17.9504,104.45,33.5406,426.8,15.9736,1.4508
2,2023-10-27,237.75,19.6028,104.40,33.4355,424.4,16.3945,1.4478
3,2023-10-30,241.30,19.3503,104.75,33.1531,428.2,15.9477,1.4623
4,2023-10-31,233.85,19.2518,106.90,32.7788,430.6,15.7948,1.4489
...,...,...,...,...,...,...,...,...
248,2024-10-21,272.90,17.7575,126.65,31.1486,526.2,14.1699,0.6532
249,2024-10-22,272.20,18.2963,126.95,30.9890,520.4,14.7228,0.6544
250,2024-10-23,277.20,17.6974,126.05,31.3139,519.6,14.0134,0.6448
251,2024-10-24,279.80,17.7883,128.45,31.6005,520.6,14.4540,0.6213


In [3]:
combined_df = retrieve_data()
np.random.seed(42)
n_days = 252
dt = 1 / n_days
close = combined_df[['ROG_Last_Price', 'CFR_Last_Price', 'ZURN_Last_Price']]
tickers = close.columns
log_returns = np.log((close / close.shift(1)).dropna())

# Heston Stochastic Volatility Model 

only use constant interest rate r + does not incorporate option prices into the calibration process.

!!! calibrate based solely on historical stock price data, particularly the realized volatility of returns.

options data need to be collect more on daily basis from 2024/25/07 to 024/25/10.
i only collected on quaterly basis so now cannot right

In [82]:
vol = log_returns.rolling(window=30).var() * n_days
vol


,ROG_Last_Price,CFR_Last_Price,ZURN_Last_Price
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
...,...,...,...
248,0.020141,0.118830,0.008371
249,0.019935,0.118104,0.009412
250,0.022582,0.118542,0.009246
251,0.018368,0.120691,0.009082


In [83]:
vol.mean()

ROG_Last_Price     0.040053
CFR_Last_Price     0.080205
ZURN_Last_Price    0.019759
dtype: float64

In [84]:
vol.iloc[0]

ROG_Last_Price    NaN
CFR_Last_Price    NaN
ZURN_Last_Price   NaN
Name: 1, dtype: float64

In [8]:
def objective(x):
    a = x ** 2 - 1
    return a

In [15]:
bounds = [(0.1, 10.0),  # kappa
          (0.001, 0.5),  # theta
          (0.01, 1.0),   # sigma
          (-0.99, 0.99)] # rho
            
result = minimize(objective, 
                  x0=[100],
                  bounds=[(-np.inf, np.inf)],
                  method='L-BFGS-B')

In [16]:
result

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -1.0
        x: [-2.149e-09]
      nit: 3
      jac: [ 1.110e-08]
     nfev: 12
     njev: 6
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>

In [35]:
sys.path.append('./linh')
from option import *

In [ ]:
input_directory = "../data/option_data/"
output_directory = "../data/option_data/processed"
# Process all files
process_all_option_data(input_directory, output_directory)

In [40]:
import os
os.walk(output_directory + 'cleaned')

<generator object _walk at 0x7f14d1b815d0>

In [ ]:
input_directory = "../data/option_data/"

In [52]:
files = ['rog_call.csv', 'cfr_call.csv', 'zurn_call.csv']

In [66]:
options = pd.DataFrame(columns=['maturity', 'strike', 'price', 'IV', 'day', 'month', 'year', 'product'])
for i, f in enumerate(files):
    for file in next(os.walk(input_directory + 'cleaned'))[1]:
        data = pd.read_csv(f'../data/option_data/cleaned/{file}/' + f)
        data['day'] = file[:2]
        data['month'] = file[2:4]
        data['year'] = file[4:]
        data['product'] = tickers[i]
        options = pd.concat([options, data])




/tmp/ipykernel_4759/335833525.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  options = pd.concat([options, data])


In [71]:
options = options.sort_values(['product', 'month', 'day', 'maturity'])

In [74]:
options

,maturity,strike,price,IV,day,month,year,product
0,0.156057,124,9.674999,24.402237,25,07,2024,CFR_Last_Price
1,0.156057,126,8.200000,23.955641,25,07,2024,CFR_Last_Price
2,0.156057,128,6.850000,23.466326,25,07,2024,CFR_Last_Price
3,0.156057,130,5.600000,22.889894,25,07,2024,CFR_Last_Price
4,0.156057,132,4.515000,22.497936,25,07,2024,CFR_Last_Price
...,...,...,...,...,...,...,...,...
49,0.229979,520,13.550000,14.122618,25,10,2024,ZURN_Last_Price
50,0.229979,540,5.600000,13.379794,25,10,2024,ZURN_Last_Price
51,0.229979,560,1.720000,12.748696,25,10,2024,ZURN_Last_Price
52,0.229979,580,1.505000,16.218067,25,10,2024,ZURN_Last_Price


In [110]:
from lmfit import Parameters, minimize

In [142]:
options[options['product']=='CFR_Last_Price']

,maturity,strike,price,IV,day,month,year,product
0,0.156057,124,9.674999,24.402237,25,07,2024,CFR_Last_Price
1,0.156057,126,8.200000,23.955641,25,07,2024,CFR_Last_Price
2,0.156057,128,6.850000,23.466326,25,07,2024,CFR_Last_Price
3,0.156057,130,5.600000,22.889894,25,07,2024,CFR_Last_Price
4,0.156057,132,4.515000,22.497936,25,07,2024,CFR_Last_Price
...,...,...,...,...,...,...,...,...
43,0.229979,130,7.125000,30.834446,25,10,2024,CFR_Last_Price
44,0.229979,135,5.060000,30.493761,25,10,2024,CFR_Last_Price
45,0.229979,140,3.480000,30.214775,25,10,2024,CFR_Last_Price
46,0.229979,145,2.350000,30.146441,25,10,2024,CFR_Last_Price


In [152]:
combined_df.iloc[0]['Date'].strftime('%Y-%m-%d')

'2023-10-25'

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

class HestonModelCalibration:
    def __init__(self, current_id, data, n_days=252):
        self.current_id = current_id
        self.data = data
        self.n_days = n_days
        self.dt = 1 / n_days
        self.tickers = data.columns
        self.log_returns = np.log((data / data.shift(1)).dropna())
        
    def calculate_realized_vol(self, window_size=30):
        """Calculate realized volatility for initial variance"""
        return self.log_returns.rolling(window=window_size).var() * self.n_days
    
    def estimate_correlation_matrix(self, window_size=30):
        """Estimate correlation matrix from historical data"""
        return self.log_returns.iloc[-window_size:].corr().values
    def func_heston(self, s, St, K, r, T, sigma, kappa, theta, volvol, rho):
        i = complex(0,1)

        prod = rho * sigma * i * s

        d1 = (prod - kappa) ** 2
        d2 = sigma ** 2 * (i * s)
        d = (d1 + d2) ** 0.5
        g1 = kappa - prod - d
        g2 = kappa - prod + d
        g = g1 / g2 if g2!=0 else np.inf

        exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
        exp2 = 1 - g * np.exp(-d * T)
        exp3 = 1 - g 
        mainExp1 = exp1 * (exp2 / exp3) ** (-2 * theta * kappa / (sigma ** 2)) if exp3!=0 else np.inf
        exp4 = theta * kappa * T / (sigma ** 2)
        # logger_yq.info(f"Calculated exp4: {exp4}")

        try:
            exp5 = volvol / (sigma ** 2)
            # logger_yq.info(f"Calculated exp5: {exp5}")
        except ZeroDivisionError:
            # logger_yq.error("Error in calculating exp5: Division by zero due to sigma being zero")
            exp5 = np.inf  # or handle it in a way that suits your application
        except Exception as e:
            # logger_yq.error(f"Unexpected error in calculating exp5: {e}")
            exp5 = np.nan  # Handle unexpected errors

        try:
            exp6_denominator = 1 - g * np.exp(-d * T)
            exp6 = (1 - np.exp(-d * T)) / exp6_denominator
            # logger_yq.info(f"Calculated exp6: {exp6}")
        except ZeroDivisionError:
            # logger_yq.error("Error in calculating exp6: Division by zero")
            exp6 = np.inf  # or handle it appropriately
        except Exception as e:
            # logger_yq.error(f"Unexpected error in calculating exp6: {e}")
            exp6 = np.nan  # Handle unexpected errors

        mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))
        # logger_yq.info(f"Calculated mainExp2: {mainExp2}")

        result = mainExp1 * mainExp2
        # logger_yq.info(f"Final result: {result}")
        
        return result
    def price_heston_mid(self, St, K, r, T, sigma, kappa, theta, volvol, rho):
        i = complex(0,1)

        P, iterations, maxNumber = 0, 1000, 100
        ds = maxNumber / iterations
        element1 = 0.5 * (St - K * np.exp(-r * T))

        for j in range(1, iterations):
            s1 = ds * (2 * j + 1) / 2
            s2 = s1 - i
            
            numerator1 = self.func_heston(s2, St, K, r, T, sigma, kappa, theta, volvol, rho)
            numerator2 = K * self.func_heston(s1, St, K, r, T, sigma, kappa, theta, volvol, rho)
            denominator = np.exp(np.log(K + 1e-8) * i * s1) * i * s1 + 1e-8
            P += ds * (numerator1 - numerator2) / denominator

        element2 = P / np.pi
        result = np.real((element1 + element2))

        return result

        
    def calibrate_heston_parameters(self, window_size=30):
        """
        Calibrate Heston model parameters using historical data
        Returns dict with parameters for each asset
        """
        model_params = {}
        for i, ticker in enumerate(self.tickers):
            # Initial estimates based on historical data
            # vol = self.calculate_realized_vol()[ticker].dropna()
            
            # def objective(x):
            #     kappa, theta, sigma, rho = x
            #     var_pred = theta + (vol.iloc[0] - theta) * np.exp(-kappa * np.arange(len(vol)) * self.dt)
            #     # ko tune cho sigma rho
            #     return np.mean((var_pred - vol)**2)
            # # variance_paths[k, i, j] = (v_prev + 
            # #             kappa[k] * (theta[k] - v_prev) * dt + 
            # #             sigma[k] * np.sqrt(v_prev * dt) * dw_s[k * 2 + 1, j] * chol_corr[k * 2 + 1][k * 2 + 1])
            # bounds = [(0.1, 10.0),  # kappa
            #          (0.001, 0.5),  # theta
            #          (0.01, 1.0),   # sigma
            #          (-0.99, 0.99)] # rho
            option_date = options['year'].astype(str) + '-' + options['month'].astype(str) + '-' + options['day'].astype(str)
            date = combined_df.iloc[self.current_id]['Date'].strftime('%Y-%m-%d')
            volatility_surface = options[(options['product']==ticker) & (option_date==date)]
            maturities = volatility_surface['maturity'].astype('float')
            strikes = volatility_surface['strike'].astype('float')
            price = volatility_surface['price'].astype('float')
            rates = 0.01107
                    
            params = Parameters()
            params.add('kappa', value=0.1, min = 0.001, max = 10)
            params.add('theta', value=0.1, min = 0.001, max = 0.1)
            params.add('volvol', value=0.1, min = 0.001, max = 0.1)
            params.add('rho', value=0.1, min = -1, max = 1.0)
            params.add('sigma', value=0.1, min = 0.001, max = 0.1)

            objective = lambda paramVect: (price - self.price_heston_mid(self.data.iloc[self.current_id][ticker], 
                                                                        strikes,  
                                                                        rates, 
                                                                        maturities, 
                                                                        paramVect['sigma'].value,                         
                                                                        paramVect['kappa'].value,
                                                                        paramVect['theta'].value,
                                                                        paramVect['volvol'].value,
                                                                        paramVect['rho'].value))/(price + 1e-8)

            result = minimize(objective, 
                              params,
                              method='leastsq',
                              max_nfev=10,
                              ftol=1e-6)
            dicts = result.params.valuesdict()
            model_params[ticker] = {
                'kappa': dicts['kappa'],
                'theta': dicts['theta'],
                'sigma': dicts['sigma'],
                'rho': dicts['rho']
            }
        return model_params

def MultivariateHestonSimulation(
    s0,
    tickers,
    dt,
    calibrated_params,
    correlation_matrix,
    n_paths=1000,
    last_id=287,
    current_id=187,
    window_size=30,
    variance_reduction='av',
    r=0.01
    ):
    
    n_assets = len(tickers)
    T = dt * (last_id - current_id)
    n_steps = int(T / dt)
    
    # Initialize arrays
    result = np.zeros((n_assets, n_paths, n_steps))
    variance_paths = np.zeros((n_assets, n_paths, n_steps))
    
    # Extract parameters for each asset
    kappa = np.array([calibrated_params[t]['kappa'] for t in tickers])
    theta = np.array([calibrated_params[t]['theta'] for t in tickers])
    sigma = np.array([calibrated_params[t]['sigma'] for t in tickers])
    rho = np.array([calibrated_params[t]['rho'] for t in tickers])
    
    # Get initial variances
    v0 = {t: calibrated_params[t]['theta'] for t in tickers}
    print("v0:",v0)
    # Ensure correlation matrix is 2D numpy array
    if isinstance(correlation_matrix, pd.DataFrame):
        correlation_matrix = correlation_matrix.values
        
    # Cholesky decomposition for correlated Brownian motions
    
        # s1 s2 s3
#     s1  1  s1s2 s1s3
#     s2 s1s2 1  s2s3
#     s3

#     correlation matrix 6x6 (only si-sj and si-vi non-zero)
#     s1          v1          s2          v2          s3          v3
# s1   1          rho_s1v1    rho_s1s2    0           rho_s1s3    0
# v1   rho_s1v1   1           rho_s2v1
# s2   rho_s1s2   rho_s2v1    1
# v2   0
# s3   rho_s1s3
# v3   0

# ==> simulate s1, s2, s3, v1, v2, v3 with 6 different z
    correlation_matrix = np.array([[1, rho[0], correlation_matrix[0][1], 0, correlation_matrix[0][2], 0],
                                       [rho[0], 1, 0, 0, 0, 0],
                                       [correlation_matrix[0][1], 0, 1, rho[1], correlation_matrix[1][2], 0],
                                       [0, 0, rho[1], 1, 0, 0],
                                       [correlation_matrix[0][2], 0, correlation_matrix[1][2], 0, 1, rho[2]],
                                       [0, 0, 0, 0, rho[2], 1]]) 
    
    try:
        chol_corr = np.linalg.cholesky(correlation_matrix)
    except np.linalg.LinAlgError:
        # If matrix is not positive definite, adjust it
        correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2  # Ensure symmetry
        min_eig = np.min(np.real(np.linalg.eigvals(correlation_matrix)))
        if min_eig < 0:
            correlation_matrix -= min_eig * np.eye(correlation_matrix.shape[0])

    
    paths_to_simulate = n_paths // 2 if variance_reduction == 'av' else n_paths
    
    for i in tqdm(range(paths_to_simulate)):
        # Generate and correlate Brownian motions
        dw_s = np.random.normal(size=(n_assets * 2, n_steps))
        
        # Correlate price and variance Brownian motion

        for j in range(n_steps):
            for k in range(n_assets):
                if j == 0:
                    result[k, i, j] = s0[tickers[k]]
                    variance_paths[k, i, j] = v0[tickers[k]]
                    
                    if variance_reduction == 'av':
                        result[k, n_paths - i - 1, j] = s0[tickers[k]]
                        variance_paths[k, n_paths - i - 1, j] = v0[tickers[k]]
                else:
                    v_prev = max(variance_paths[k, i, j-1], 0)
                    
                    # Update variance
                    result[k, i, j] = result[k, i, j-1] * np.exp(
                        (r - 0.5 * v_prev) * dt + 
                        np.sqrt(v_prev * dt) * dw_s[k * 2, j] * chol_corr[k * 2][k * 2])
                    
                    variance_paths[k, i, j] = (v_prev + 
                        kappa[k] * (theta[k] - v_prev) * dt + 
                        sigma[k] * np.sqrt(v_prev * dt) * dw_s[k * 2 + 1, j] * chol_corr[k * 2 + 1][k * 2 + 1])
                    
                    # Update price
                    
                    if variance_reduction == 'av':
                        result[k, n_paths - i - 1, j] = result[k, n_paths - i - 1, j-1] * np.exp(
                        (r - 0.5 * v_prev) * dt + 
                        np.sqrt(v_prev * dt) * (-dw_s[k * 2, j]))
                    
                        variance_paths[k, n_paths - i - 1, j] = (v_prev + 
                            kappa[k] * (theta[k] - v_prev) * dt + 
                            sigma[k] * np.sqrt(v_prev * dt) * (-dw_s[k * 2 + 1, j]))
    
    if variance_reduction == 'emc':
        for k in range(n_assets):
            correction_factor = result[k][-1, :].mean() / result[k][-1, :]
            result[k] = result[k] * correction_factor
    
    return result, tickers, variance_paths

def run_heston_backtest(combined_df, backtest_start, backtest_end, last_id):
    # Initialize model and calibrate parameters
    close = combined_df[['ROG_Last_Price', 'CFR_Last_Price', 'ZURN_Last_Price']]
    
    prices = []
    for current_id in tqdm(range(backtest_start, backtest_end)):
        # Get data for current window
        window_data = close.iloc[max(0, current_id - 30):current_id]
        heston_calibration = HestonModelCalibration(current_id, close)
        calibrated_params = heston_calibration.calibrate_heston_parameters(window_size=30)
        correlation_matrix = heston_calibration.estimate_correlation_matrix(window_size=30)
        
        payoff = []
        result, _, variance_paths = MultivariateHestonSimulation(
            s0=close.iloc[current_id],
            tickers=close.columns,
            dt=1/252,
            calibrated_params=calibrated_params,
            correlation_matrix=correlation_matrix,
            n_paths=10,
            variance_reduction='av',
            current_id=current_id,
            last_id=last_id
        )
        
        # Construct paths and calculate payoffs
        old_path_rog = combined_df['ROG_Last_Price'].iloc[:current_id]
        old_path_cfr = combined_df['CFR_Last_Price'].iloc[:current_id]
        old_path_zurn = combined_df['ZURN_Last_Price'].iloc[:current_id]

        for i in range(result.shape[1]):
            path_rog = np.append(old_path_rog, result[0][i].flatten())
            path_cfr = np.append(old_path_cfr, result[1][i].flatten())
            path_zurn = np.append(old_path_zurn, result[2][i].flatten())
        

            # print(path_cfr)
            # print(path_zurn)
            
        #     payoff.append(payoff_func(
        #         path_rog, path_cfr, path_zurn, 
        #         combined_df.iloc[current_id]['Date'], 
        #         verbose=False
        #     ))
        
        # prices.append(neutral_pricing(
        #     np.mean(payoff), 
        #     0.01107, 
        #     last_id - current_id
        # ))
            payoff.append(payoff_func(path_rog, path_cfr, path_zurn, combined_df.iloc[current_id]['Date'], risk_free=0.01107, risk_neutral=True, verbose=False))
        # print(path_rog)

            # print(payoff_func(path_rog, path_cfr, path_zurn, combined_df.iloc[current_id]['Date'], risk_free=0.01107, risk_neutral=True, verbose=False))
        prices.append(np.mean(payoff))
        
    return prices

In [178]:
import matplotlib.pyplot as plt

def evaluation_plot(backtest_start, backtest_end, combined_df, prices):
    # Load and process actual price data
    real_price = pd.read_csv('../data/product_price_full.csv')
    real_price['Date'] = pd.to_datetime(real_price['Date'], dayfirst=True)
    real_price = real_price.sort_values(by='Date', ascending=True)
    real_price['Product'] = 1000 * real_price['Product']

    # Filter actual prices for the backtest period
    actual = real_price[(real_price['Date'] >= combined_df.iloc[backtest_start]['Date']) & 
                        (real_price['Date'] <= combined_df.iloc[backtest_end - 1]['Date'])]['Product'].values
    dates = real_price[(real_price['Date'] >= combined_df.iloc[backtest_start]['Date']) & 
                       (real_price['Date'] <= combined_df.iloc[backtest_end - 1]['Date'])]['Date']

    # Ensure `prices` has the same length as `dates`
    if len(prices) != len(dates):
        prices = np.mean(prices, axis=0) if prices.ndim > 1 else np.repeat(prices, len(dates))

    # Plotting
    plt.figure(figsize=(20, 4))
    plt.plot(dates, prices, marker='o', label='Predicted Prices')
    plt.plot(dates, actual, marker='o', label='Actual Prices')
    plt.title('Price Comparison Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.ylim(850, 1150)
    plt.legend()
    plt.show()



In [ ]:
# Initialize backtest parameters
last_id = 287
backtest_start = 187
backtest_end = 188

# Run backtest
prices = run_heston_backtest(
    combined_df,
    backtest_start,
    backtest_end,
    last_id
)

# Evaluate results
evaluation_plot(
    backtest_start,
    backtest_end,
    combined_df,
    prices
)


  0%|          | 0/3 [01:48<?, ?it/s]


TypeError: 'MinimizerResult' object is not subscriptable

In [179]:
np.append(old, new.flatten())

array([237.55      , 237.25      , 237.75      , 241.3       ,
       233.85      , 237.7       , 238.15      , 237.25      ,
       237.15      , 236.4       , 238.15      , 238.1       ,
       236.75      , 238.75      , 239.6       , 238.        ,
       236.85      , 237.6       , 237.15      , 237.6       ,
       236.6       , 238.35      , 238.7       , 236.9       ,
       235.9       , 236.4       , 236.05      , 239.6       ,
       246.3       , 250.95      , 249.95      , 249.45      ,
       253.3       , 254.3       , 253.7       , 252.65      ,
       245.9       , 244.35      , 245.2       , 244.05      ,
       243.5       , 245.        , 244.3       , 243.7       ,
       242.45      , 244.5       , 251.85      , 254.35      ,
       251.3       , 251.45      , 252.25      , 253.9       ,
       250.6       , 252.5       , 250.85      , 250.1       ,
       247.25      , 245.95      , 244.1       , 247.8       ,
       243.75      , 245.15      , 242.        , 247.45